AMBIENTE

In [ ]:
%%bash
# Instalar miniconda e mamba (padrão já visto em aula)
wget --quiet https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh
bash Miniforge3-Linux-x86_64.sh -b -p /usr/local -u
mamba init
mamba config set auto_activate_base true
mamba shell init --shell bash --root-prefix=~/.local/share/mamba
rm Miniforge3-Linux-x86_64.sh

In [ ]:
%%bash
# Instalar os software que vamos usar nesta prática
mamba install -c bioconda -c conda-forge fastqc bwa gatk4 cutadapt samtools bedtools ensembl-vep=115 -y

In [ ]:
%%bash
# Testa se um deles foi instalado corretamente (deve printar o manual de uso do programa)
vep --help

In [ ]:
%%bash
# Criar as pastas para organizar os arquivos que serão gerados
mkdir -p raw fastqc cutadapt bwa gatk vep reference

In [ ]:
%%bash
# Mover arquivos para as suas respectivas pastas e descompactar fasta.gz
mv *.fastq.gz raw/
gunzip -c Chr12-reference.fasta.gz > reference/Chr12-reference.fasta
rm -rf Chr12-reference.fasta.gz sample_data/

Controle de Qualidade

In [ ]:
%%bash
# Gerar o relatório da qualidade dos reads de sequenciamento
fastqc -o fastqc/ /content/raw/caso2-sindrome-metabolica_R1.fastq.gz /content/raw/caso2-sindrome-metabolica_R2.fastq.gz

In [ ]:
%%bash
# Filtrar sequências muito pequenas e cortar as pontas
cutadapt \
  -u 5 -U 5 \
  -u -5 -U -5 \
  -m 90 \
  -o cutadapt/caso2-sindrome-metabolica-trimmed_R1.fastq.gz \
  -p cutadapt/caso2-sindrome-metabolica-trimmed_R2.fastq.gz \
  raw/caso2-sindrome-metabolica_R1.fastq.gz \
  raw/caso2-sindrome-metabolica_R2.fastq.gz

Mapeamento com a referência do genoma humano

In [ ]:
%%bash
# Indexar o genoma humano de referência para realizar o mapeamento e posterior remoção de contaminante (cerca de 3 min)
samtools faidx reference/Chr12-reference.fasta
bwa index reference/Chr12-reference.fasta

In [ ]:
%%bash
# Proceder com o alinhamento contra o genoma humano (cerca de 2 min)
bwa mem reference/Chr12-reference.fasta \
  -R '@RG\tID:sample1\tSM:sample1\tPL:ILLUMINA\tLB:lib1\tPU:unit1' \
  cutadapt/caso2-sindrome-metabolica-trimmed_R1.fastq.gz \
  cutadapt/caso2-sindrome-metabolica-trimmed_R2.fastq.gz \
  | samtools sort -o bwa/caso2-sindrome-metabolica.bam -T sample1 -@ 2

In [ ]:
%%bash
# Indexar o bam para gerar o bai e visualizar no IGV
samtools index bwa/caso2-sindrome-metabolica.bam

Chamada de Variantes

In [ ]:
%%bash
# Indexar a referência e BAM para chamada de variantes com GATK
samtools dict reference/Chr12-reference.fasta > reference/Chr12-reference.dict
samtools index bwa/caso2-sindrome-metabolica.bam

In [ ]:
%%bash
# Chamada de variantes
gatk HaplotypeCaller \
  -R reference/Chr12-reference.fasta \
  -I bwa/caso2-sindrome-metabolica.bam \
  -O gatk/caso2-sindrome-metabolica_variants.vcf

Anotação de Variantes

In [ ]:
%%bash
# script oficial baixa cache do genoma humano (grande volume de dados, demora cerca de 60 min)
vep_install -a cf -s homo_sapiens -y GRCh38 --NO_UPDATE

In [ ]:
%%bash
# Index VCF files para anotação
bgzip gatk/caso2-sindrome-metabolica_variants.vcf
tabix -p vcf gatk/caso2-sindrome-metabolica_variants.vcf.gz

In [ ]:
%%bash
# Anotação de variantes a nível de proteína
vep -i gatk/caso2-sindrome-metabolica_variants.vcf.gz \
    --output_file vep/caso2-sindrome-metabolica_variants.vep.tsv \
    --tab --force_overwrite \
    --offline --cache --dir_cache ~/.vep \
    --assembly GRCh38 \
    --symbol --hgvs --canonical --af_1kg --max_af \
    --everything --pick\
    --stats_file vep/caso2-sindrome-metabolica_variants.vep.html

RESPOSTA: A partir dos arquivos .tsv e .html gerados pelo VEP, foi possível identificar a presença da predição probably damaging e localizar a variante correspondente NM_000277.3(PAH):c.734T>C (p.Val245Ala), uma mutação missense. Em seguida, utilizei o banco ClinVar para verificar a relevância clínica da variante, confirmando sua associação com a Fenilcetonúria (PKU).
O gene PAH é conhecido por estar relacionado a um fenótipo autossômico recessivo, e a variante analisada é classificada como patogênica, corroborando sua relação direta com o desenvolvimento da PKU.